In [112]:
from collections import OrderedDict
from math import log, sqrt

import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_file

dataCSV = 'updatedsi.csv'
df = pd.read_csv(dataCSV)
print(df.head(3))

               name  year status gender
0   Lobsang Kelsang  2011  Alive   Male
1  Kelsang Wangchuk  2011  Alive   Male
2      Dawa Tsering  2011  Alive   Male


In [113]:
gender_color = OrderedDict([
    ("M",'#f0e1d2'),# "#8F0505"),
    ("F", "#c64737"),
])

gend_color = {
    'Male': '#ffd23f',#153B50',#FAF33E',#FAA916',#930B08',
    'Female': '#2ab7ca',#FAF33E',#187795',#613984',
}
#FF8000
#8F0505
status_color = {
    "Unknown" : '#1e441e',#E5F77D',#FBBA72',#EC9F05',#'#BC0101',#'#823038',#'#fdae61',#"orange",
    "Alive" : '#38A700',#E5F77D',#930B08',#D76A03',#D63B29',#'#BC0101',#'#fb6a4a'-pinkish red,#abd9e9' -blue,#'green',#"#e69584",
    "Dead" : 'red',#930B08',#3E000C',#'black',#'#FF0000',#'#BC0101'-not darkenough red,#fb6a4a',#'red',#"#aeaeb8", #d73027
}

width = 800
height = 825
inner_radius = 90
outer_radius = 300 - 10
extra_radius = 20

maxr = 16
minr = 8
b = (outer_radius - inner_radius) / (maxr - minr)
a = b * maxr - inner_radius

big_angle = 2.0 * np.pi / (len(df))
small_angle = big_angle / 7

x = np.zeros(len(df))
y = np.zeros(len(df))

output_file("si.html", title="si")

p = figure(plot_width=width, plot_height=height, title="Self Immolations in Tibet since 2009",
    x_axis_type=None, y_axis_type=None,
    x_range=[-420, 420], y_range=[-420, 420],
    min_border=0, outline_line_color="white",
    background_fill="white", border_fill="white")
#f0e1d2
p.line(x+1, y+1, alpha=0)

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [status_color[status] for status in df.status]
p.annular_wedge(
    x, y, inner_radius, outer_radius+extra_radius, -big_angle+angles, angles, color=colors,
)

color_genders = [gend_color[gender] for gender in df.gender]
# small wedges
#p.annular_wedge(x, y, inner_radius, rad(df.year-2000),
#p.annular_wedge(x, y, inner_radius+(22*rad(df.year-2008)),4,-big_angle+angles+5*small_angle,-big_angle+angles+6*small_angle,color=gender_color['M'])
# p.annular_wedge(x, y, inner_radius+22,rad(df.year-2000),-big_angle+angles+5*small_angle,
#                 -big_angle+angles+6*small_angle,color=gender_color['M'])

def rad(mic):
    return abs((b * mic)-a)
#     #return b * np.sqrt(np.log(mic * 1E4)) + a

#*******************************************
#*******************************************
#p.annular_wedge(x, y,inner_radius, rad(df.year-2008)-inner_radius,-big_angle+angles+5*small_angle,
#                 -big_angle+angles+6*small_angle,color=gender_color['M'])
# p.annular_wedge(x, y,outer_radius - rad(df.year-2008),a-rad(df.year-2008),-big_angle+angles+5*small_angle,
#                 -big_angle+angles+6*small_angle,color=gender_color['M'])
p.annular_wedge(x, y,40+inner_radius+outer_radius-rad(df.year-2008),44+inner_radius+a-rad(df.year-2008),-big_angle+angles+5*small_angle,
                -big_angle+angles+6*small_angle, color=color_genders)#color=gender_color[df.gender[:]])
#*******************************************
#*******************************************
#Labels - names
#radii[6] gives inner most circle
xr = (outer_radius+extra_radius*1.7)*np.cos(np.array(-big_angle/2 + angles))
yr = (outer_radius+extra_radius*1.7)*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
#label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.name, angle=label_angle, text_font_size="6pt", text_align="left", 
       text_baseline="middle", text_color='#8F0505')

# circular axes and labels
labels=np.arange(2008,2016)
radii = a - b * np.arange(1,9)
p.circle(x, y, radius=radii, fill_color=None, line_color="white")
# p.text(x[:]-10, radii[::-1]+5, [str(r) for r in labels[:]],text_font_size="13pt", text_align="center",
#        text_baseline="middle", text_color='black')#, text_font_style='')
p.text(x[:]-5, radii[:]-413, [str(r) for r in labels[:]],text_font_size="9pt", text_align="center",
       text_baseline="middle", text_color='white', text_font_style='bold')

#Inner circle legends - for status
p.circle([-20,-20,-20],[21, 0, -21], color=list(status_color.values()), radius=9) # radius of legend
p.text([-5, -5, -5], [28, 6, -16], text=[gr for gr in status_color.keys()],
    text_font_size="11pt", text_align="left", text_baseline="top")
#gend_colors
p.rect([-45,20],[-50, -50], color=list(gend_color.values()), height=16, width=10) # radius of legend
p.text([-35,26],[-43, -43], text=[gr for gr in gend_color.keys()],
    text_font_size="10pt", text_align="left", text_baseline="top")

# bottom legends - for gender
# p.circle([-40, -40, -40], [-350, -370,-390], radius=8, color=list(status_color.values()))
# p.text([-40, -40, -40], [-350, -370, -390], text=list(status_color.keys()),
#     text_font_size="9pt", text_align="left", text_baseline="middle")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

show(p)

#print(df)

ERROR:C:\Anaconda3\lib\site-packages\bokeh\validation\check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 00d54a88-d693-4688-8bc9-40e261aa60eb
ERROR:C:\Anaconda3\lib\site-packages\bokeh\validation\check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: a4548f73-d7be-494b-a6a6-07fbbdcd61c3


In [ ]:
drug_color = OrderedDict([
    ("Year",   "#0d3362"),
    ("Gender", "#c64737"),
    ("Neomycin",     "black"  ),
])

gram_color = {
    "Dead" : "#aeaeb8",
    "Alive" : "#e69584",
    "Unknown" : "orange",
}

width = 800
height = 800
inner_radius = 90
outer_radius = 300 - 10

# minr = sqrt(log(.001 * 1E4))
# maxr = sqrt(log(1000 * 1E4))

# y = ax + b
maxr = 16
minr = 7
a = (outer_radius - inner_radius) / (maxr - minr)
b = inner_radius - a * maxr # b = y - a(x)
#b = inner_radius - a * maxr

def rad(mic):
    return a * (mic) + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

x = np.zeros(len(df))
y = np.zeros(len(df))

output_file("burtin.html", title="si.py example")

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=[-420, 420], y_range=[-420, 420],
    min_border=0, outline_line_color="black",
    background_fill="#f0e1d2", border_fill="#f0e1d2")

p.line(x+1, y+1, alpha=0)

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [gram_color[gram] for gram in df.gram]
p.annular_wedge(
    x, y, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# small wedges
p.annular_wedge(x, y, inner_radius, rad(df.year-2000),
    -big_angle+angles+5*small_angle, -big_angle+angles+6*small_angle,
    color=drug_color['Year'])
#Needs to be a float
# p.annular_wedge(x, y, inner_radius, rad(df.gender),
#     -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
#     color=drug_color['Gender'])

# circular axes and lables


# radial axes
p.annular_wedge(x, y, inner_radius-10, outer_radius+10,
    -big_angle+angles, -big_angle+angles, color="black")


# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-40, -40], [-370, -390], color=list(gram_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["Status-" + gr for gr in gram_color.keys()],
    text_font_size="7pt", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
    color=list(drug_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(drug_color.keys()),
    text_font_size="9pt", text_align="left", text_baseline="middle")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

show(p)


In [61]:
gender_color = OrderedDict([
    ("Male",   "#0d3362"),
    ("Female", "#c64737"),
])

gram_color = {
    "Alive" : "#aeaeb8",
    "Dead" : "#e69584",
    "Unknown" : 'orange'
}

dataCSV = 'antibiotics.csv'
#df = pd.read_csv(StringIO(antibiotics),skiprows=1,skipinitialspace=True,engine='python')
df = pd.read_csv(dataCSV)

width = 800
height = 800
inner_radius = 90
outer_radius = 300 - 10

# minr = sqrt(log(.001 * 1E4))
# maxr = sqrt(log(1000 * 1E4))
maxr = 16
minr = 8
#a = (outer_radius - inner_radius) / (maxr-minr)
# a= outer_radius
# b = inner_radius
#b = inner_radius - a * maxr

# def rad(mic):
#     return (a * mic) + b

def rad(mic):
    return (mic)

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

#returns a new array of given shape and type, filled with zeros.
x = np.zeros(len(df))
#y = np.zeros(len(df))
y = np.arange(2008,2016)

output_file("si.html", title="si.py example")

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=[-420, 420], y_range=[-420, 420],
    min_border=0, outline_line_color="black",
    background_fill="#f0e1d2", border_fill="#f0e1d2")

p.line(x+1, y+1, alpha=0)

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [gram_color[gram] for gram in df.gram]
p.annular_wedge(
    x, y, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# small wedges
p.annular_wedge(x, y, inner_radius, rad(df.penicillin),
    -big_angle+angles+2*small_angle, -big_angle+angles+3*small_angle,
    color=gender_color['Penicillin'])
# p.annular_wedge(x, y, inner_radius, rad(df.streptomycin),
#     -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
#     color=drug_color['Streptomycin'])
# p.annular_wedge(x, y, inner_radius, rad(df.neomycin),
#     -big_angle+angles+1*small_angle, -big_angle+angles+2*small_angle,
#     color=drug_color['Neomycin'])

# circular axes and lables
#labels = np.power(10.0, np.arange(-3, 4))
#radii = a * np.sqrt(np.log(labels * 1E4)) + b
labels = np.arange(2008,2016)
radii = minr * np.sqrt(labels) + maxr

p.circle(x, y, radius=radii, fill_color=None, line_color="white")
#p.text(x[:-1], radii[:-1], [str(r) for r in labels[:-1]],
p.text(x[1:], radii[1:], [str(r) for r in labels[1:]],
       text_font_size="8pt", text_align="center", text_baseline="middle")

print(radii[1:])
#[str(r) for r in labels[:1]],
# radial axes
# initially inner_radius-10, outer_radius+10
p.annular_wedge(x, y, inner_radius+10, outer_radius-10,
    -big_angle+angles, -big_angle+angles, color="black")

# bacteria labels
# martyr names
# xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
# yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
xr = radii[0]
yr = radii[0]
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.name, angle=label_angle,
    text_font_size="9pt", text_align="center", text_baseline="middle")

# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-40, -40], [-370, -390], color=list(gram_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["Current Status -" + gr for gr in gram_color.keys()],
    text_font_size="7pt", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
    color=list(drug_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(drug_color.keys()),
    text_font_size="9pt", text_align="left", text_baseline="middle")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

show(p)

AttributeError: 'DataFrame' object has no attribute 'penicillin'